In [69]:
import pyiast
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

**Aiida Lab data importation**

In [93]:
#entre liste des pression et liste des loading_absolute_average, les transfo en csv file puis les importe sur ce doc
def conv_to_csv(pressure,loading_absolute_average,mof_gas):
    df = pd.DataFrame({
        'Pressure (bar)': pressure,
        'Loading (mmol/g)': loading_absolute_average
    })

    # Save data into a csv file
    file_name = f'isotherm_data_{mof_gas}.csv'
    df.to_csv(file_name, index=False)
    return

    #import data from the csv file
def import_data(mof_gas):
    file_name = f'isotherm_data_{mof_gas}.csv'
    df_mof = pd.read_csv(file_name)
    return df_mof

**Loading q values in pure conditions**

In [94]:
def get_pure_q(df_mof):
    pure_isotherm = pyiast.InterpolatorIsotherm(df_mof,
                                        loading_key="Loading (mmol/g)",
                                        pressure_key="Pressure (bar)")
    pure_isotherm = pyiast.InterpolatorIsotherm(df_mof,
                                        loading_key="Loading (mmol/g)",
                                        pressure_key="Pressure (bar)",
                                        fill_value=df_mof['Loading (mmol/g)'].max())
    return pure_isotherm

**Loading q values in binary mixture**

In [95]:
def get_mixt_q(N2_isotherm, CO2_isotherm, pressure):
    p_part = pressure*np.array([0.85,0.15])
    q_mix = pyiast.iast(p_part, [N2_isotherm, CO2_isotherm], verboseflag=False)
    return q_mix

**selectivity and q_mix calculation**

In [96]:
def get_selectivity(mof_N2,mof_CO2, S_pressure):
    name_N2 = f"{mof_N2}"
    name_CO2 = f"{mof_CO2}"

    df_N2 = import_data(name_N2)
    df_CO2 = import_data(name_CO2)
    
    N2_isotherm = get_pure_q(df_N2)
    CO2_isotherm = get_pure_q(df_CO2)
    
    q_mix = get_mixt_q(N2_isotherm, CO2_isotherm, S_pressure)
    Selectivity_mof = (0.85/0.15)*(q_mix[1]/q_mix[0])
    return Selectivity_mof , q_mix

**for NOTT**

In [97]:

P_list_CO2 =  [0.2, 0.3266840603101, 0.48686234189789, 0.69500812430223, 0.97485889138887, 1.3678130038057, 1.951904804549, 2.551904804549, 3.151904804549, 3.751904804549, 4.351904804549, 4.951904804549, 5.551904804549, 6.151904804549, 6.751904804549, 7.351904804549, 7.951904804549, 8.551904804549, 9.151904804549, 9.751904804549, 10.2]
q_mean_CO2=[2.732727414792, 4.369949526912, 5.7705666370224, 6.8743309833244, 7.598813085141, 8.2150826016624, 8.6262039121069, 8.846552379985, 9.0288549244554, 9.1648323824958, 9.2861528044446, 9.3393688103836, 9.4082578858388, 9.4298963790073, 9.4631941067159, 9.5820096542812, 9.5691030123974, 9.6485346916435, 9.603024443808, 9.6529810951878, 9.7229767415754]
P_list_N2 = [0.2, 0.8, 1.4, 2.0, 2.6, 3.2, 3.8, 4.4, 5.0, 5.6, 6.2, 6.8, 7.4, 8.0, 8.6, 9.2, 9.8, 10.2]
q_mean_N2 = [0.19363986752364, 0.719462293293, 1.1852683313032, 1.5837143737829, 1.964938147429, 2.3003800603546, 2.6374516505899, 2.9101006884275, 3.1565843516202, 3.4335188371982, 3.664822358103, 3.8379403631471, 4.031771365684, 4.2347466686208, 4.4040319141264, 4.5423130517804, 4.6845174866617, 4.786422617184]



mof_CO2 = "NOTT_300_CO2"
mof_N2 = "NOTT_300_N2"


fichier_CO2 = conv_to_csv(P_list_CO2,q_mean_CO2,mof_CO2)
fichier_N2 = conv_to_csv(P_list_N2,q_mean_N2,mof_N2)

S_pressure = 0.2
S , q_mix_NOTT_02 = get_selectivity(mof_N2,mof_CO2, S_pressure)
q_mix_NOTT_CO2_02=q_mix_NOTT_02[1]
print(S)
print(q_mix_NOTT_02)
print(q_mix_NOTT_CO2_02) #q of N2 and then q of CO2

14.531504153242693
[0.15820241 0.40569159]
0.4056915919660641


In [98]:


S_pressure = 1
S , q_mix_NOTT_1 = get_selectivity(mof_N2,mof_CO2, S_pressure)
q_mix_NOTT_CO2_1=q_mix_NOTT_1[1]
print(S)
print(q_mix_NOTT_1)
print(q_mix_NOTT_CO2_1) #q of N2 and then q of CO2


16.52407564396486
[0.67507112 1.96851641]
1.9685164086523286


**Calculation of Working Capacity of NOTT**

In [99]:
W_NOTT = q_mix_NOTT_CO2_1 - q_mix_NOTT_CO2_02
print(W_NOTT)

1.5628248166862644


**for Mg MOF**

In [103]:
P_list_CO2 =  [0.2, 0.8, 1.4, 2.0, 2.6, 3.2, 3.8, 4.4, 5.0, 5.6, 6.2, 6.8, 7.4, 8.0, 8.6, 9.2, 9.8, 10.2]
q_mean_CO2=[0.23575968759626, 0.98475549405786, 1.8503006720399, 2.7077428446667, 3.6901292397011, 4.821734538441, 5.8649964779462, 6.635211171313, 7.4943700823624, 8.2962757670132, 8.5992285964759, 9.203984040658, 9.7064218748443, 10.027881144504, 10.348482044953, 10.553271771403, 10.764379095248, 10.953048647908]
P_list_N2 = [0.2, 0.8, 1.4, 2.0, 2.6, 3.2, 3.8, 4.4, 5.0, 5.6, 6.2, 6.8, 7.4, 8.0, 8.6, 9.2, 9.8, 10.2]
q_mean_N2 = [0.052583697888832, 0.20784552084234, 0.35462665598747, 0.5127725994912, 0.6547296996684, 0.78499581118667, 0.91900441246653, 1.055776962607, 1.2148671122433, 1.3078284978352, 1.4631418229414, 1.5659572870799, 1.6924809088365, 1.8241547458615, 1.885013122948, 1.9804466118686, 2.1478114407025, 2.2276397773604]


mof_CO2 = "Mg_MOF_CO2"
mof_N2 = "Mg_MOF_N2"


fichier_CO2 = conv_to_csv(P_list_CO2,q_mean_CO2,mof_CO2)
fichier_N2 = conv_to_csv(P_list_N2,q_mean_N2,mof_N2)


S_pressure = 0.2
S , q_mix_Mg_02 = get_selectivity(mof_N2,mof_CO2, S_pressure)
q_mix_Mg_CO2_02=q_mix_Mg_02[1]
print(S)
print(q_mix_Mg_02)
print(q_mix_Mg_CO2_02)#q of N2 then q of CO2

4.488274064277231
[0.04453978 0.03527766]
0.035277663422569015


In [106]:
 
S_pressure = 1
S , q_mix_Mg_1 = get_selectivity(mof_N2,mof_CO2, S_pressure)
q_mix_Mg_CO2_1=q_mix_Mg_1[1]
print(S)
print(q_mix_Mg_1)
print(q_mix_Mg_CO2_1)
#q of N2 and then q of CO2

4.585586617478118
[0.21943318 0.17756997]
0.1775699742401944


**Calculation of Working Capacity of Mg**

In [107]:
W_Mg = q_mix_Mg_CO2_1 - q_mix_Mg_CO2_02
print(W_Mg)

0.1422923108176254


**for UTSA-20**

In [108]:
P_list_CO2 =  [0.2, 0.66381721559481, 1.2298724660926, 1.8298724660926, 2.4298724660926, 3.0298724660926, 3.6298724660926, 4.2298724660926, 4.8298724660926, 5.4298724660926, 6.0298724660926, 6.6298724660926, 7.2298724660926, 7.8298724660926, 8.4298724660926, 9.0298724660926, 9.6298724660926, 10.2]
q_mean_CO2=[0.63946236515945, 1.9724895914484, 3.4569226858202, 4.4732069487837, 5.3521324574436, 6.0205913102688, 6.579024095708, 7.0017784828232, 7.3607290084119, 7.669732401093, 8.0171687632951, 8.3256175557268, 8.492933001113, 8.7782044696897, 9.0469538174041, 9.2581902914954, 9.4641622020254, 9.7410898340143]
P_list_N2 = [0.2, 0.8, 1.4, 2.0, 2.6, 3.2, 3.8, 4.4, 5.0, 5.6, 6.2, 6.8, 7.4, 8.0, 8.6, 9.2, 9.8, 10.2]
q_mean_N2 = [0.08188217413345, 0.32251246144232, 0.53957637696006, 0.76012516867171, 0.95660265403681, 1.1491068838839, 1.3278371612686, 1.4974372285776, 1.6924826864331, 1.8655014090106, 2.0024876706094, 2.1327525082901, 2.3114248423652, 2.4386021294038, 2.5679647069774, 2.7012177639123, 2.8451820555145, 2.8981836286015]


mof_CO2 = "UTSA_20_CO2"
mof_N2 = "UTSA_20_N2"


fichier_CO2 = conv_to_csv(P_list_CO2,q_mean_CO2,mof_CO2)
fichier_N2 = conv_to_csv(P_list_N2,q_mean_N2,mof_N2)


S_pressure = 0.2
S , q_mix_UTSA20_02 = get_selectivity(mof_N2,mof_CO2, S_pressure)
q_mix_UTSA20_CO2_02=q_mix_UTSA20_02[1]
print(S)
print(q_mix_UTSA20_02)
print(q_mix_UTSA20_CO2_02)#q of N2 then q CO2

7.834786517779227
[0.06916688 0.09563078]
0.09563077934408999


In [109]:

S_pressure = 1
S , q_mix_UTSA20_1 = get_selectivity(mof_N2,mof_CO2, S_pressure)
q_mix_UTSA20_CO2_1=q_mix_UTSA20_1[1]
print(S)
print(q_mix_UTSA20_1)
print(q_mix_UTSA20_CO2_1)
#q of N2 and then q of CO2

8.084929809435833
[0.32583453 0.46488517]
0.46488516709640537


**Calculation of Working Capacity of UTSA-20**

In [110]:
W_UTSA_20 = q_mix_UTSA20_CO2_1 - q_mix_UTSA20_CO2_02
print(W_UTSA_20)

0.3692543877523154


**for UTSA 80**

In [112]:
P_list_CO2 =  [0.2, 0.8, 1.4, 2.0, 2.6, 3.2, 3.8, 4.4, 5.0, 5.6, 6.2, 6.8, 7.4, 8.0, 8.6, 9.2, 9.8, 10.2]
q_mean_CO2=[0.31424485381695, 1.1580909367748, 2.038693985348, 2.9664843410745, 3.8041249233618, 4.7398294176147, 5.6300892453135, 6.571769721719, 7.5454467796638, 8.1926294954818, 9.0655394233608, 9.9333829424732, 10.499469115282, 11.176310724031, 11.800907117546, 12.322322186213, 12.73597406036, 13.056237399758]
P_list_N2 = [0.2, 0.8, 1.4, 2.0, 2.6, 3.2, 3.8, 4.4, 5.0, 5.6, 6.2, 6.8, 7.4, 8.0, 8.6, 9.2, 9.8, 10.2]
q_mean_N2 = [0.06563379544785, 0.26439683199255, 0.45816146525745, 0.62400984619095, 0.8023627360098, 0.97143287688045, 1.1356066227015, 1.3174702957764, 1.4491204175354, 1.6241835419303, 1.7920465854102, 1.9268249595619, 2.1054243692705, 2.239310171408, 2.337518595033, 2.5546176109562, 2.6749108164195, 2.7594671385695]

mof_CO2 = "UTSA_80_CO2"
mof_N2 = "UTSA_80_N2"


fichier_CO2 = conv_to_csv(P_list_CO2,q_mean_CO2,mof_CO2)
fichier_N2 = conv_to_csv(P_list_N2,q_mean_N2,mof_N2)


S_pressure = 0.2
S , q_mix_UTSA80_02 = get_selectivity(mof_N2,mof_CO2, S_pressure)
q_mix_UTSA80_CO2_02=q_mix_UTSA80_02[1]
print(S)
print(q_mix_UTSA80_02)
print(q_mix_UTSA80_CO2_02)#q of N2 then q of CO2

4.784444117141291
[0.05591037 0.04720589]
0.047205893280265004


In [113]:

S_pressure = 1
S , q_mix_UTSA80_1 = get_selectivity(mof_N2,mof_CO2, S_pressure)
q_mix_UTSA80_CO2_1=q_mix_UTSA80_1[1]
print(S)
print(q_mix_UTSA80_1)
print(q_mix_UTSA80_CO2_1)
#q of N2 then q of CO2

4.7365211033586565
[0.2721755 0.2274997]
0.22749970482841725


**Calculation of Working Capacity of UTSA-80**

In [114]:
W_UTSA_80 = q_mix_UTSA80_CO2_1 - q_mix_UTSA80_CO2_02
print(W_UTSA_80)

0.18029381154815224


**RECAP OF THE RESULTS**

In [136]:
BOLD = "\033[1m"
RESET = "\033[0m"


MOF = ["NOTT_300_CO2", "Mg_MOF_CO2", "UTSA_20_CO2", "UTSA_80_CO2"]  
WC = [1.5628248166862644, 0.1422923108176254, 0.3692543877523154, 0.18029381154815224]  
binary_loading_adsorption = [1.9685164086523286, 0.1775699742401944, 0.46488516709640537, 0.22749970482841725]  
binary_loading_desorption = [0.4056915919660641,0.035277663422569015, 0.09563077934408999 , 0.047205893280265004]

column1 = ["MOF"] + MOF


print(f"{BOLD}{column1[0]:<20} {'WC':<15}{'q adsorption':<25}{'q desorption':<25}{RESET}")


for item1, wc_value, bin_load_ads, bin_load_des in zip(column1[1:], WC, binary_loading_adsorption, binary_loading_desorption):
    print(f"{item1:<20} {round(wc_value, 2):<15} {round(bin_load_ads, 2):<25} {round(bin_load_des, 2):<25}")


MOF                  WC             q adsorption             q desorption             
NOTT_300_CO2         1.56            1.97                      0.41                     
Mg_MOF_CO2           0.14            0.18                      0.04                     
UTSA_20_CO2          0.37            0.46                      0.1                      
UTSA_80_CO2          0.18            0.23                      0.05                     
